In [1]:
import numpy as np
import scipy.stats as ss
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import tensorflow as tf
import tensorflow.experimental.numpy as tnp
import tensorflow_probability as tfp
import scipy.special

tfd = tfp.distributions
np.random.seed(0)

from LSM_tensorflow import *

2024-01-30 03:34:17.143081: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-30 03:34:17.145380: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 03:34:17.188326: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 03:34:17.188347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 03:34:17.189618: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
job_name = 'complete'

In [3]:
S0 = 27479.363
sigma = .1797
_lambda = 11.9488

eta1 = 1/0.074
eta2 = 1/0.0608
p = .3316
r = 0.01
d = 0.2

n = 200_000 # sample size
m = 360 # sample length
T = 1 #years

LTV0 = 0.6
LTVH = 0.8
liquidation_penalty = 0.95

degree = 2
seed = 0
print(n, m, LTV0, LTV0/LTVH)

200000 360 0.6 0.7499999999999999


In [4]:
LTVH = 0.9

start = -0.1
stop  = 0.8
step  = .05
all_d_arr = np.arange(start, stop+step, step)

start = 0.1
stop  = 0.8
step  = .05
LTV0_arr = np.arange(start, stop+step, step)

start = 10
stop  = m
step  = 10
slice_id_arr = np.arange(start, stop+step, step)

print(LTVH, all_d_arr, LTV0_arr, slice_id_arr)

0.9 [-0.1  -0.05  0.    0.05  0.1   0.15  0.2   0.25  0.3   0.35  0.4   0.45
  0.5   0.55  0.6   0.65  0.7   0.75  0.8 ] [0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75
 0.8  0.85] [ 10  20  30  40  50  60  70  80  90 100 110 120 130 140 150 160 170 180
 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360]


In [5]:
import pickle
with open(r"results1.pickle", "rb") as input_file:
    results1 = pickle.load(input_file)
    
with open(r"results2.pickle", "rb") as input_file:
    results2 = pickle.load(input_file)
        
with open(r"results3.pickle", "rb") as input_file:
    results3 = pickle.load(input_file)
    
all_results = results1 + results2 + results3

In [6]:
finished_jobs = []

for a in all_results:
    finished_jobs.append((a[0],a[1],a[2]))
    
finished_jobs

[(-0.1, 0.1, 10),
 (-0.1, 0.1, 20),
 (-0.1, 0.1, 30),
 (-0.1, 0.1, 40),
 (-0.1, 0.1, 50),
 (-0.1, 0.1, 60),
 (-0.1, 0.1, 70),
 (-0.1, 0.1, 80),
 (-0.1, 0.1, 90),
 (-0.1, 0.1, 100),
 (-0.1, 0.1, 110),
 (-0.1, 0.1, 120),
 (-0.1, 0.1, 130),
 (-0.1, 0.1, 140),
 (-0.1, 0.1, 150),
 (-0.1, 0.1, 160),
 (-0.1, 0.1, 170),
 (-0.1, 0.1, 180),
 (-0.1, 0.1, 190),
 (-0.1, 0.1, 200),
 (-0.1, 0.1, 210),
 (-0.1, 0.1, 220),
 (-0.1, 0.1, 230),
 (-0.1, 0.1, 240),
 (-0.1, 0.1, 250),
 (-0.1, 0.1, 260),
 (-0.1, 0.1, 270),
 (-0.1, 0.1, 280),
 (-0.1, 0.1, 290),
 (-0.1, 0.1, 300),
 (-0.1, 0.1, 310),
 (-0.1, 0.1, 320),
 (-0.1, 0.1, 330),
 (-0.1, 0.1, 340),
 (-0.1, 0.1, 350),
 (-0.1, 0.1, 360),
 (-0.1, 0.15000000000000002, 10),
 (-0.1, 0.15000000000000002, 20),
 (-0.1, 0.15000000000000002, 30),
 (-0.1, 0.15000000000000002, 40),
 (-0.1, 0.15000000000000002, 50),
 (-0.1, 0.15000000000000002, 60),
 (-0.1, 0.15000000000000002, 70),
 (-0.1, 0.15000000000000002, 80),
 (-0.1, 0.15000000000000002, 90),
 (-0.1, 0.150000000

In [7]:
_Kou_model_params = Kou_model_params(S0, sigma, _lambda, eta1, eta2, p, r, T)

d_arr = all_d_arr

for d in d_arr:
    _price_paths_simulation = price_paths_simulation(seed, n, m, _Kou_model_params, d)
    _contract_params =  contract_params(_price_paths_simulation.d, LTV0, LTVH, _Kou_model_params, liquidation_penalty)
    _price_paths_simulation.gen_price_paths()

    for LTV0 in LTV0_arr:
        _contract_params_job = contract_params(_price_paths_simulation.d, LTV0, LTVH, _Kou_model_params, _contract_params.liquidation_penalty)

        payoffs, healthy_mask = gen_payoffs_and_healthy_mask(_contract_params_job, _price_paths_simulation.price_t,
                                                            _price_paths_simulation.path_mins)

        _LSM_regression = LSM_regression(_price_paths_simulation, 
                                        payoffs,
                                        _price_paths_simulation.price_t, 
                                        healthy_mask, degree, _contract_params)

        for slice_id in slice_id_arr:
            if (d, LTV0, slice_id) in finished_jobs:
                continue
            try:
                _LSM_regression.run_regression(slice_id, 1, False)
                
                _mean = _LSM_regression.A.mean()
                _var  = _LSM_regression.A.var()
                
                all_results.append([d, LTV0,  slice_id, _mean, _var])
            except:
                all_results.append([d, LTV0,  slice_id, 'error', 'error'])
                
            print('%.2f %.2f %i'%(d, LTV0,  slice_id))
            
        with open('results_toComplete.pickle', 'wb') as handle:
            pickle.dump(all_results, handle)
            print('saved')

Price paths (price_t) and running minimums (path_mins) are generated.
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
Price paths (price_t) and running minimums (path_mins) are generated.
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
Price paths (price_t) and running minimums (path_mins) are generated.
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
Price paths (price_t) and running minimums (path_mins) are generated.
saved
saved
saved
saved
saved
saved
saved
saved
0.05 0.50 10
0.05 0.50 20
0.05 0.50 30
0.05 0.50 40
0.05 0.50 50
0.05 0.50 60
0.05 0.50 70
0.05 0.50 80
0.05 0.50 90
0.05 0.50 100
0.05 0.50 110
0.05 0.50 120
0.05 0.50 130
0.05 0.50 140
0.05 0.50 150
0.05 0.50 160
0.05 0.50 170
0.05 0.50 180
0.05 0.50 190
0.05 0.50 200
0.05 0.50 210
0.05 0.50 220
0.05 0.50 230
0.05 0.50 240
0.05 0.50 250
0.05 0.50 260
0.05 0.50 270
0.05 0.50 280
0